In [47]:
import requests
import json
import pandas as pd
from pprint import pprint
from time import sleep
import os

from spotify_credentials import client_id, client_secret

In [60]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_key = r['access_token']



In [61]:
songdf = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\master_data_fullList.csv")
songdf["explicit"] = False

songdf = songdf.iloc[:,[1,2,3,4,5,6,7,8,9,10,11]]
#songdf.head()
songdf = songdf.head(2000)

In [62]:
#VERSION TO PICK UP WHERE SPOTIFY CUT ME OFF

spotify_endpoint = "https://api.spotify.com/v1/tracks/"
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_key)}

for index, row in songdf.iterrows():

    song_id = str(songdf.iat[index,0])

    try:
        spotify_response = requests.get((spotify_endpoint + song_id), headers = spotify_headers).json()
        explicit = spotify_response["explicit"]
        songdf.iat[index,10] = explicit
        sleep(0.25)
        
    except:
        if spotify_response == {'error': {'status': 429}}:
            print(f"{spotify_response} on song ID = {song_id}")
        else:
            print(f"Failure on ID: {song_id}")
        pass

        



songdf.head()


path = r"C:\Users\kevin\bootcamp\Project_1\resources"
songdf.to_csv(os.path.join(path,f"master_data_test.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas